In [ ]:
from opensearchpy import OpenSearch
import pandas as pd
from datetime import datetime

In [ ]:
my_datetime = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
print(my_datetime)

my_datetime = datetime(2022, 2, 18, 12, 30, 0)
print(my_datetime)

## Connect to OpenSearch
Set the API endpoint URL, the search query and any required headers or parameters. Set the authentication credentials

In [ ]:
client = OpenSearch(
    hosts = [{"host": "localhost", "port": 9200}],
    http_auth = ("admin", "admin"),
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)
client.info()

## Create an Index

In [ ]:
index_name = 'search_index'
body = {
  "mappings": {
    "properties": {
      "doc_name": {
        "type": "text",
        "analyzer": "edge_ngram_analyzer"
      },
      "doc_type": {
        "type": "text",
        "analyzer": "standard"
      },
      "link": {
        "type": "text",
        "analyzer": "standard"
      },
      "source": {
        "type": "text",
        "analyzer": "standard"
      },
      "created_date": {
        "type" : "date",
        "format" : "strict_date_time_no_millis"
      },
      "modified_date": {
        "type" : "date",
        "format" : "strict_date_time_no_millis"
      },
      "summary": {
        "type" : "text",
        "analyzer": "standard"
      },
      "file_size": {
        "type" : "integer",
      }
    }
  },
  "settings": {
    "analysis": {
      "analyzer": {
        "edge_ngram_analyzer": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": ["lowercase", "edge_ngram_filter"]
        }
      },
      "filter": {
        "edge_ngram_filter": {
          "type": "edge_ngram",
          "min_gram": 2,
          "max_gram": 10
        }
      }
    }
  }
}
response = client.indices.create(index=index_name, body=body)

## Add data to index

First check the number of items in the index.

In [ ]:
doc_name = "Google"
doc_type = "txt"
link = "https://www.google.com/"
source = "Gmail"
created_date = datetime(2022, 2, 18, 12, 30, 0).strftime('%Y-%m-%dT%H:%M:%SZ')
modified_date = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
summary = "Google is a multinational technology company that specializes in Internet-related services and products. It was founded in 1998 by Larry Page and Sergey Brin, who were graduate students at Stanford University at the time. Today, Google is one of the largest and most influential companies in the world, with a market capitalization of over $1 trillion."
file_size = len(summary.encode('utf-8'))
item1 = {
    "doc_name": doc_name,
    "doc_type": doc_type,
    "link": link,
    "source": source,
    "created_date": created_date,
    "modified_date": modified_date,
    "summary": summary,
    "file_size": file_size
}


doc_name = "Youtube"
doc_type = "txt"
link = "https://www.google.com/"
source = "Google drive"
created_date = datetime(2022, 2, 18, 12, 30, 0).strftime('%Y-%m-%dT%H:%M:%SZ')
modified_date = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
summary = "YouTube is a video-sharing website and online platform that allows users to upload, share, and view videos. It was founded in 2005 by three former PayPal employees, and was later acquired by Google in 2006. Today, YouTube is one of the largest websites in the world, with over 2 billion monthly active users and over 1 billion hours of video watched daily."
file_size = len(summary.encode('utf-8'))

item2 = {
    "doc_name": doc_name,
    "doc_type": doc_type,
    "link": link,
    "source": source,
    "created_date": created_date,
    "modified_date": modified_date,
    "summary": summary,
    "file_size": file_size
}
to_be_inserted = []
to_be_inserted.append(item1)
to_be_inserted.append(item2)
print(to_be_inserted)

In [ ]:
response = client.count(index=index_name)
id = int(response['count'])
print(id)

In [ ]:
for body in to_be_inserted:
    client.index(index=index_name, id=str(id),body=body)
    id += 1

In [ ]:
response = client.count(index=index_name)
id = int(response['count'])
print(id)

## Search Data

In [ ]:

response = client.search(
    index=index_name,
    body={
        "query": {
            "match": {
                "doc_name": "Youtube"
            }
        }          
    }
)

response = client.search(
    index=index_name,
    body={
        "query": {
            "match_all": {}
        }        
    }
)

print(response)
print(type(response))

## Delete document from index

In [ ]:
client.delete(index = index_name, id = "1")

## Delete an index

In [ ]:
client.indices.delete(index=index_name)